<center><img src="../../fig/ICHEC_Logo.jpg" alt="Drawing" style="width: 500px;"/>

# <center>Understanding the Global Interpreter Lock</center>
***

In [1]:
import threading

We have two simple functions that will print out strings of different colour a total of 5 times.

In [2]:
def red_text():
    c = '\033[91m'  # red
    for i in range(5):
        for word in ['Hello', 'from', 'the', 'first', 'red', 'function\n']:
            print(f' {c}{word}', end='')

def blue_text():
    c = '\033[34m' 
    for i in range(5):
        for word in ['Hello', 'from', 'the', 'second', 'blue', 'function\n']:
            print(f' {c}{word}', end='')

Using the module, `threading` which we have imported, we define 2 threads. We will run it and then they will join together to the main thread.

In [13]:
red_thread = threading.Thread(target=red_text)
blue_thread = threading.Thread(target=blue_text)

red_thread.start()
blue_thread.start()

red_thread.join()
blue_thread.join()

 Hello from the first red function
 Hello from the first red function
 Hello from the first red function
 Hello from the first red function
 Hello Hello from the second blue function
 Hello from from the first the second blue function
 Hello from the second blue function
 Hello from the second red function
 blue function
 Hello from the second blue function


As you can see, they are interfering with each other and it will always be different no matter how many times you do it.

Here the GIL is running and everything is protected but we see the threads interfering over the loop.

Now lets have a look at the lock

In [14]:
lock = threading.Lock()

In [15]:
def red_text():
    c = '\033[91m'  # red
    for i in range(5):
        lock.acquire()
        for word in ['Hello', 'from', 'the', 'first', 'red', 'function\n']:
            print(f' {c}{word}', end='')
        lock.release()

def blue_text():
    c = '\033[34m' 
    for i in range(5):
        lock.acquire()
        for word in ['Hello', 'from', 'the', 'second', 'blue', 'function\n']:
            print(f' {c}{word}', end='')
        lock.release()

The parts that are interfering we are wrapping by the lock.

In [20]:
red_thread = threading.Thread(target=red_text)
blue_thread = threading.Thread(target=blue_text)

red_thread.start()
blue_thread.start()

red_thread.join()
blue_thread.join()

 Hello from the first red function
 Hello from the first red function
 Hello from the first red function
 Hello from the first red function
 Hello from the first red function
 Hello from the second blue function
 Hello from the second blue function
 Hello from the second blue function
 Hello from the second blue function
 Hello from the second blue function


Remember this itself is not the GIL, but this is how the GIL works. For every line of of code the interpreter will translate it to machine code, and then get wrapped by the lock.

To have a truly multithreaded code you need to release the GIL. We will look at this in more detail in the `numba` section.